# AMLD 2020: Transfer Learning for International Crisis Response

In [ ]:
import os
import pandas as pd

In [ ]:
train = [pd.read_csv("org" + str(i) + "_dev.csv") for i in range (1, 5)]

In [ ]:
test = [pd.read_csv("org" + str(i) + "_test.csv") for i in range(1, 5)]

In [ ]:
for i in range(4):
    train[i]["labels"] = pd.Series(map(lambda x: x.split(";"), train[i]["labels"]))

In [ ]:
map_labels = {
    101: 10,
    102: 6,
    103: 3,
    104: 4,
    105: 10,
    106: 5,
    107: 7,
    108: 10,
    109: 9,
    110: 10,
    111: [8, 11],
    112: 12
}

In [ ]:
reversemap_labels = {
    1: 104,
    2: 108,
    3: 103,
    4: 104,
    5: 106,
    6: 102,
    7: 107,
    8: 111,
    9: 109,
    10: 110,
    11: 111,
    12: 112
}

In [ ]:
def encode_labels(labels):
    labels = [int(i) - 1 for i in labels]
    multi_hot = [0 for i in range(12)]
    for i in range(12):
        if i in labels:
            multi_hot[i] = 1
    return multi_hot

In [ ]:
a = train[3]["labels"]
for i in a:
    if "111" in i:
        i.remove("111")
        i.append(8)
        i.append(11)
    for j in range(len(i)):
        if int(i[j]) in map_labels.keys():
            i[j] = map_labels[int(i[j])]
train[3]["labels"] = a

In [ ]:
for i in range(4):
    train[i]["labels"] = pd.Series(map(lambda x: encode_labels(x), train[i]["labels"]))

In [ ]:
for i in range(4):
    train[i]["text"] = train[i]["entry_translated"].apply(lambda x: x.replace("\n", " "))

In [ ]:
train_all = pd.concat([train[i] for i in range(4)]).reset_index(drop=True)

In [ ]:
from simpletransformers.classification import MultiLabelClassificationModel
model = MultiLabelClassificationModel('roberta', 'roberta-base', num_labels=12, args={'output_dir': 'output-1234/', 'cache_dir': 'cache-1234/', 'n_gpu': 1, 'train_batch_size':2, 'gradient_accumulation_steps':16, 'learning_rate': 3e-5, 'num_train_epochs': 3, 'max_seq_length': 512})

In [ ]:
model.train_model(train_all)

In [ ]:
test_all = pd.concat([test[i] for i in range(4)]).reset_index(drop=True)
to_predict = test_all["entry_translated"].apply(lambda x: x.replace("\n", " ")).tolist()
preds, outputs = model.predict(to_predict)
sub_df = pd.DataFrame(outputs, columns=[str(i) for i in range(1, 13)])
sub_df["id"] = test_all["id"]
sub_df.to_csv("sub_all_n.csv")

In [ ]:
import numpy as np

In [ ]:
sub_arr = np.array(sub_df.drop('id', axis=1))
preds = list(map(lambda x: np.argmax(x), sub_arr))
pred_dict = {sub_df["id"][i]: preds[i] for i in range(len(preds))}
for i in pred_dict.keys():
    pred_dict[i] += 1
    if "org4" in i:
        pred_dict[i] = reversemap_labels[pred_dict[i]]

In [ ]:
submission = pd.DataFrame()
submission["id"] = pd.Series(list(pred_dict.keys()))
submission["predicted_label"] = pd.Series(map(lambda x: pred_dict[x], submission["id"]))

In [ ]:
submission.to_csv("submission_all_n.csv", index=False)